In [ ]:
from database import init, Post, load_sqlite_as_df
from tqdm import tqdm
from data_loader import pull_posts
import peewee

# init()  # This will NUKE the local database

In [ ]:
df = load_sqlite_as_df()
resume_from = 1 + (len(df) / 10)
print(f"Resuming from {resume_from}")

In [ ]:
def ingest(resume_from: int = 0) -> None:
    """Ingest data to local database."""

    i = int(resume_from)
    batch_size = 10
    with tqdm() as progress_bar:
        # Start from `resume_from`
        progress_bar.update(i * batch_size)
        while True:
            try:
                parsed_posts = pull_posts(i, batch_size=batch_size, order="asc")
                Post.bulk_create(
                    list(map(lambda x: Post(**x), parsed_posts)), batch_size=batch_size
                )
            except peewee.IntegrityError:
                print("Duplicate entry")
            finally:
                i += 1
                progress_bar.update(batch_size)